<a href="https://colab.research.google.com/github/ericaerin/GE120/blob/main/ME4_AY23-24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

\*For easier discussion of the codes for the class, the notebook lacks script and function docstrings *italicized text*

# imports

In [ ]:
import math
from google.colab import files

#in case pandas is not installed
try:
  import pandas as pd
except ModuleNotFoundError:
  !pip install pandas==1.5.3
finally:
  import pandas as pd

# file check

In [ ]:
def file_to_list(file_path: str) -> list:
  with open(file_path, 'r') as data:
    datalist = data.readlines() #store file rows to a list

  return datalist

In [ ]:
def file_check(file_path: str) -> bool:
  try:
    #check for correct file format
    assert(file_path.split('.')[-1] == 'txt'), "File not .TXT"

    #check for correct header
    filelist = file_to_list(file_path)
    assert(filelist[0].replace('\n', '').split('\t') == ['Line', 'Distance', 'BearingNS',
                                                         'BearingDeg', 'BearingMin', 'BearingSec', 'BearingEW']), "Incorrect header."

    #check for correct per-column value types
    temp_df = pd.read_csv(file_path, delimiter='\t')
    assert(temp_df['Distance'].dtype == 'float64'), "Distance column has incorrect values."
    assert((sorted(temp_df['BearingNS'].dropna().unique()) == ['N', 'S'])), "BearingNS column has incorrect values."
    assert((sorted(temp_df['BearingEW'].dropna().unique()) == ['E', 'W'])), "BearingEW column has incorrect values."
    assert(temp_df['BearingDeg'].dtype == 'int64'), "BearingDeg column has incorrect values."
    assert(temp_df['BearingMin'].dtype == 'int64'), "BearingMin column has incorrect values."
    assert(temp_df['BearingSec'].dtype == 'float64'), "BearingSec column has incorrect values."

  #return boolean True or False for looping
  except FileNotFoundError:
    print("File not found.")
    return True
  except AssertionError as error:
    print(error)
    return True
  else:
    return False

# get Point Name

In [ ]:
def getPointName(line: str) -> str:
  return line.split('-')[-1] #get point in consideration based on preprocessing done

# read file and convert to dataframe

In [ ]:
def preprocess(file_path: str):
  dictionary = {}
  datalist = file_to_list(file_path)[1:] #do not get header anymore
  header = ['Line', 'Distance', 'BearingNS', 'BearingDeg', 'BearingMin', 'BearingSec', 'BearingEW']

  for string in datalist: #loop through each row
    delimited = string.replace('\n', '').split("\t") #remove \n and create list with items per column
    dictionary[delimited[0]] = {header[1] : delimited[1],
                                header[2] : delimited[2],
                                header[3] : delimited[3],
                                header[4] : delimited[4],
                                header[5] : delimited[5],
                                header[6] : delimited[6]}

  #make dataframe from dictionary ensuring correct row items
  lines_dataframe = pd.DataFrame.from_dict(dictionary, orient='index') #keys (Lines) made into index
  lines_dataframe.reset_index(inplace=True) #Lines made to a new column
  lines_dataframe.rename(columns={'index' : 'Line'}, inplace=True) #rename appropriate column name

  #assign dataframe data types
  lines_dataframe = lines_dataframe.astype({'Line':'string', 'Distance':float,
                                            'BearingNS':'string', 'BearingDeg':int,
                                            'BearingMin':int, 'BearingSec':float, 'BearingEW':'string'})

  #get point name using function
  lines_dataframe['Point'] = lines_dataframe.apply(lambda x: getPointName(x['Line']), axis=1)

  return lines_dataframe

# compute latitude and departure

In [ ]:
def getLat(distance: float, bearing: tuple):
  #compute for bearing decimal degrees
  n_s, deg, min, sec, e_w = bearing
  angle = deg + (min/60) + (((sec/60)) / 60)

  #compute latitude based on N/S/None orientation
  if n_s == 'N':
    latitude = distance * math.cos(math.radians(angle))
  elif n_s == 'S':
    latitude = -distance * math.cos(math.radians(angle))
  elif n_s == 'None':
    latitude = 0
  else:
    latitude = None

  return latitude

In [ ]:
def getDep(distance: float, bearing: tuple):
  #compute for bearing decimal degrees
  n_s, deg, min, sec, e_w = bearing
  angle = deg + (min/60.) + ((sec/60.))/60.

  #compute departure based on N/S/None orientation
  if e_w == 'E':
    departure = distance * math.sin(math.radians(angle))
  elif e_w == 'W':
    departure = -distance * math.sin(math.radians(angle))
  elif e_w == 'None':
    departure = 0
  else:
    departure = None

  return departure

# compute Northings and Eastings

In [ ]:
def getCoords(dataframe, northings, eastings): #more for per item/row than whole column processing
  for i in range(len(dataframe)):
    if i == 0:
      dataframe.loc[i, 'Northings'] = northings + dataframe.loc[i, 'Latitude'] #loc[row identifier, column identifier] to change the 'cell' value
      dataframe.loc[i, 'Eastings'] = eastings + dataframe.loc[i, 'Departure']
    else:
      dataframe.loc[i, 'Northings'] = dataframe.loc[i-1, 'Northings'] + dataframe.loc[i, 'Latitude']
      dataframe.loc[i, 'Eastings'] = dataframe.loc[i-1, 'Eastings'] + dataframe.loc[i, 'Departure']

  return dataframe

# compute REC and LEC

In [ ]:
def computeEC(sumLat: float, sumDep: float) -> tuple:
  lec = math.sqrt((sumLat**2) + (sumDep**2))

  while True:
    try:
      rec = lec/float(input("Input sum of distance here: "))
      break
    except ZeroDivisionError:
      print("Inputted distance is 0, please input correct nonzero distance.")
    except ValueError:
      print("Inputted distance is not a number, please input correct nonzero distance.")

  if rec <= 1/100000:
    rec_desc = 'First Order Geodetic Control'
  elif rec <= 1/50000:
    rec_desc = 'Second Order Geodetic Control'
  elif rec <= 1/20000:
    rec_desc = 'Third Order Geodetic Control / Primary Project Control'
  elif rec <= 1/10000:
    rec_desc = 'Fourth Order Geodetic Control / Secondary Project Control'
  elif rec <= 1/5000:
    rec_desc = 'Tertiary Project Control'
  else:
    rec_desc = 'No accuracy standard achieved'

  return lec, "1 : {}".format(int(((1/rec)//1000) * 1000)), rec_desc

# main

In [ ]:
#reminder for upload of input file
files.upload() #upload ME4-2324-TRAVERSE.txt

In [ ]:
#input filename and check if file contains correct data
path = input("Input file path here (Sample: /content/ME4-2324-TRAVERSE.txt): ")
while file_check(path):
  path = input("Input file path here (Sample: /content/ME4-2324-TRAVERSE.txt): ")
else:
  print("Final file path is {}".format(path))

Input file path here (Sample: /content/ME4-2324-TRAVERSE.txt): /content/ME4-2324-TRAVERSE.txt
Final file path is /content/ME4-2324-TRAVERSE.txt


In [ ]:
#create dataframe from correct file
lines_df = preprocess(path)

In [ ]:
lines_df.head() #sample run output

,Line,Distance,BearingNS,BearingDeg,BearingMin,BearingSec,BearingEW,Point
0,T1-T2,2.996054,N,52,47,9.142218,W,T2
1,T2-T3,2.307003,N,36,46,13.998929,W,T3
2,T3-T4,3.768860,N,22,46,31.431194,W,T4
3,T4-T5,136.389807,N,0,17,41.652164,E,T5
4,T5-T6,9.308655,N,74,45,10.723956,E,T6


In [ ]:
lines_df.info() #sample run output data types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Line        57 non-null     string 
 1   Distance    57 non-null     float64
 2   BearingNS   57 non-null     string 
 3   BearingDeg  57 non-null     int64  
 4   BearingMin  57 non-null     int64  
 5   BearingSec  57 non-null     float64
 6   BearingEW   57 non-null     string 
 7   Point       57 non-null     object 
dtypes: float64(2), int64(2), object(1), string(3)
memory usage: 3.7+ KB


In [ ]:
#update dataframe latitude and departure
lines_df['Latitude'] = lines_df.apply(lambda x: getLat(x['Distance'], (x['BearingNS'],
                                                       x['BearingDeg'], x['BearingMin'],
                                                       x['BearingSec'], x['BearingEW'])), axis=1)

lines_df['Departure'] = lines_df.apply(lambda x: getDep(x['Distance'], (x['BearingNS'],
                                                       x['BearingDeg'], x['BearingMin'],
                                                       x['BearingSec'], x['BearingEW'])), axis=1)

In [ ]:
lines_df.head() #sample run output with Lat and Dep

,Line,Distance,BearingNS,BearingDeg,BearingMin,BearingSec,BearingEW,Point,Latitude,Departure
0,T1-T2,2.996054,N,52,47,9.142218,W,T2,1.812,-2.386
1,T2-T3,2.307003,N,36,46,13.998929,W,T3,1.848,-1.381
2,T3-T4,3.768860,N,22,46,31.431194,W,T4,3.475,-1.459
3,T4-T5,136.389807,N,0,17,41.652164,E,T5,136.388,0.702
4,T5-T6,9.308655,N,74,45,10.723956,E,T6,2.448,8.981


In [ ]:
#compute Northing and Easting of the dataframe
final_df = getCoords(lines_df, northings=1620574.565, eastings=506576.200)

In [ ]:
final_df.head() #sample run output

,Line,Distance,BearingNS,BearingDeg,BearingMin,BearingSec,BearingEW,Point,Latitude,Departure,Northings,Eastings
0,T1-T2,2.996054,N,52,47,9.142218,W,T2,1.812,-2.386,1620576.377,506573.814
1,T2-T3,2.307003,N,36,46,13.998929,W,T3,1.848,-1.381,1620578.225,506572.433
2,T3-T4,3.768860,N,22,46,31.431194,W,T4,3.475,-1.459,1620581.700,506570.974
3,T4-T5,136.389807,N,0,17,41.652164,E,T5,136.388,0.702,1620718.088,506571.676
4,T5-T6,9.308655,N,74,45,10.723956,E,T6,2.448,8.981,1620720.536,506580.657


In [ ]:
final_df[['Point', 'Northings', 'Eastings']].to_csv('RESULTS.csv') #export results

In [ ]:
final_df['Distance'].sum() #get total distance

959.0519681170001

In [ ]:
lec_rec = computeEC(final_df['Latitude'].sum(), final_df['Departure'].sum()) #compute Error of Closures

Input sum of distance here: 959.0519681170001


In [ ]:
lec_rec #sample run output

(0.007071053513400378, '1 : 135000', 'First Order Geodetic Control')

In [ ]:
#write output README
with open('README.txt', 'w') as readme:
  readme.write("LEC: {:.4f}\n".format(lec_rec[0]))
  readme.write("REC: {}\n".format(lec_rec[1]))
  readme.write("Accuracy Assessment: {0}\n{1:-<27}\n".format(lec_rec[2], ''))
  readme.write("The traverse has {} points.\
  \nNo corrections were made with T1 backsight used: 1620574.565 N, 506576.200 E.\
  \nThe T1 coordinates found in the last row are the foresight to close the loop.".format(len(final_df['Point'])))